<a href="https://colab.research.google.com/github/damayant/Data-Science-and-ML/blob/master/llm_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install tensorflow


In [3]:
pip install nltk


In [4]:
pip install colorama


In [5]:
pip install numpy


In [7]:
pip install scikit_learn


In [8]:
pip install Flask

In [9]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [11]:
with open('intents.json') as file :
  data = json.load(file)
training_sentences = []
training_labels = []
responses = []
labels = []

for intent in data['intents']:
  for pattern in intent['patterns']:
    training_sentences.append(pattern)
    training_labels.append(intent['tag'])
  responses.append(intent['responses'])

  if intent['tag'] not in labels:
    labels.append(intent['responses'])

num_classes = len(labels)

The variable “training_sentences” holds all the training data (which are the sample messages in each intent category) and the “training_labels” variable holds all the target labels correspond to each training data.

Then we use “LabelEncoder()” function provided by scikit-learn to convert the target labels into a model understandable form.

In [12]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

Next, we vectorize our text data corpus by using the “Tokenizer” class and it allows us to limit our vocabulary size up to some defined number. When we use this class for the text pre-processing task, by default all punctuations will be removed, turning the texts into space-separated sequences of words, and these sequences are then split into lists of tokens. They will then be indexed or vectorized. We can also add “oov_token” which is a value for “out of token” to deal with out of vocabulary words(tokens) at inference time.

In [14]:

vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

The “pad_sequences” method is used to make all the training text sequences into the same size.

# **Model Training**
Let’s define our Neural Network architecture for the proposed model and for that we use the “Sequential” model class of Keras.

In [15]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16680 (65.16 KB)
Trainable params: 16680 (65.16 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

Now we are ready to train our model. Simply we can call the “fit” method with training data and labels.

In [16]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 1s 15ms/step - loss: 2.0804 - accuracy: 0.0606
Epoch 2/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0796 - accuracy: 0.1818
Epoch 3/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0797 - accuracy: 0.0909
Epoch 4/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0796 - accuracy: 0.0909
Epoch 5/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0791 - accuracy: 0.0909
Epoch 6/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0784 - accuracy: 0.0909
Epoch 7/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0775 - accuracy: 0.1212
Epoch 8/500
2/2 [==============================] - 0s 11ms/step - loss: 2.0768 - accuracy: 0.4242
Epoch 9/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0766 - accuracy: 0.3030
Epoch 10/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0764 - accuracy: 0.2121
Epoch 11/500
2/2 [=========

After training, it is better to save all the required files in order to use it at the inference time. So that we save the trained model, fitted tokenizer object and fitted label encoder object.

In [17]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

Inference
Okay!!!! now it’s time to check how our model performs. 😊

We are going to implement a chat function to engage with a real user. When a new user message is received, the chatbot will calculate the similarity between the new text sequence and training data. Considering the confidence scores got for each category, it categorizes the user message to an intent with the highest confidence score.

In [18]:

import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: Hello
1/1 [==============================] - 0s 96ms/step
ChatBot: Hi there
User: I am sad
1/1 [==============================] - 0s 35ms/step
ChatBot: Tell me your problem to assist you
User: i am overwhelmed
1/1 [==============================] - 0s 22ms/step
ChatBot: Yes Sure, How can I support you
User: how can i get better
1/1 [==============================] - 0s 21ms/step
ChatBot: Hi
User: please advise
1/1 [==============================] - 0s 21ms/step
ChatBot: Yes Sure, How can I support you
User: i am sad
1/1 [==============================] - 0s 31ms/step
ChatBot: Yes Sure, How can I support you
User: support me
1/1 [==============================] - 0s 21ms/step
ChatBot: Yes Sure, How can I support you
User: how to get calm
1/1 [==============================] - 0s 33ms/step
ChatBot: Hello
User: 

KeyboardInterrupt: ignored